In [69]:
trigger_port=['trigport0', 'trigport1']
trigger_name=['trigname0', 'trigname1']
triggertype=['trigtype0', 'trigtype1']
out_port=['outport0', 'outport1']
out_name=['outname0', 'outname1']
out_chan=[i for i,_ in enumerate(out_port)]
t_on=[1000, 2000]
t_off=[500, 1000]
t_offset=[500, 1000]
t_duration=[8000, 8000]
p_trigger=7

In [74]:
mainprog = Path('galil_toggle_main.dmc').read_text()
subprog = Path('galil_toggle_sub.dmc').read_text()

mainlines = mainprog.split("\n")
subindex = [i for i,x in enumerate(mainlines) if x.strip().startswith("XQ")][0]
subline = mainlines.pop(subindex)

for i in range(len(trigger_port)):
    mainlines.insert(subindex+i+1, subline.format(subthread=i+1))
clearbits = [i for i,x in enumerate(mainlines) if x.strip().startswith("CB")]

for i in clearbits:
    mainlines[i] = '    ' + ''.join([f'CB {oc};' for oc in out_chan])
    
haltindex = [i for i,x in enumerate(mainlines) if x.strip().startswith("HX")][0]
mainlines.pop(haltindex)
haltline = '    ' + ''.join([f'HX{i+1};' for i in range(len(out_chan))])
mainlines.insert(haltindex+1, haltline)

prog_parts = ["\n".join(mainlines).format(p_trigger=p_trigger)] + [
    subprog.format(
        subthread=i + 1,
        p_output=out_chan[i],
        t_duration=t_duration[i],
        t_offset=t_offset[i],
        t_time_on=t_on[i],
        t_time_off=t_off[i],
    )
    for i in range(len(trigger_port))
]

In [75]:
print('\n'.join(prog_parts))

#main;
    CB 0;CB 1;
    AI 7;
    AI -7;
    XQ #tgl1,1;
    XQ #tgl2,2;
    CB 0;CB 1;
    HX1;HX2;
EN;

#tgl1;
    cnt1=0;
    AT 0;CB 0;
	AT 500;cnt1=cnt1+500;
    #jt1;
    AT 0;SB 0;
    AT 1000;CB 0;cnt1=cnt1+1000;AT 0;
    IF (500>0);
        AT 500;cnt1=cnt1+500;
    ENDIF;
    JP #jt1,(cnt1<8000)|(8000<0);
    CB 0;
EN;

#tgl2;
    cnt2=0;
    AT 0;CB 1;
	AT 1000;cnt2=cnt2+1000;
    #jt2;
    AT 0;SB 1;
    AT 2000;CB 1;cnt2=cnt2+2000;AT 0;
    IF (1000>0);
        AT 1000;cnt2=cnt2+1000;
    ENDIF;
    JP #jt2,(cnt2<8000)|(8000<0);
    CB 1;
EN;

